<a href="https://colab.research.google.com/github/ShravaniMundhe04/Music-Sequence-Generator/blob/main/classical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import stream, note, midi


s = stream.Stream()


for pitch in ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']:
    n = note.Note(pitch)
    n.quarterLength = 1  # each note = 1 beat
    s.append(n)


s.write('midi', fp='simple_song.mid')




'simple_song.mid'

In [ ]:
from google.colab import files

files.download('simple_song.mid')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from music21 import converter, note, chord

song = converter.parse('simple_song.mid')

notes = []
for element in song.flat.notes:
    if isinstance(element, note.Note):
        notes.append(element.pitch.midi)
    elif isinstance(element, chord.Chord):
        notes.append(max(p.midi for p in element.pitches))

print(notes)


[60, 62, 64, 65, 67, 69, 71, 72]


/usr/local/lib/python3.12/dist-packages/music21/stream/base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [ ]:
sequence_length = 4
network_input = []
network_output = []


for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

print("Number of sequences:", len(network_input))
print("Example input:", network_input[0], "Output:", network_output[0])


Number of sequences: 4
Example input: [60, 62, 64, 65] Output: 67


In [ ]:
import numpy as np

# Convert to numpy arrays
network_input = np.array(network_input)
network_output = np.array(network_output)

# Normalize input (divide by max MIDI note, 127)
network_input = network_input / 127
network_output = network_output / 127


In [ ]:
network_input = np.reshape(network_input, (network_input.shape[0], network_input.shape[1], 1))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))  # linear because we predict a note number


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')


In [ ]:
model.fit(network_input, network_output, epochs=100, batch_size=16)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.2898
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2598
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2196
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1798
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.1528
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1140
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0791
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0553
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0239
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0103
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0019
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0118
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0127
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0425
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0171
Epoch 16/100
1/1 ━━━━

In [ ]:
start = 0  # start with first sequence
pattern = network_input[start]
prediction_output = []

# Generate 20 notes
for note_index in range(20):
    prediction = model.predict(pattern.reshape(1, pattern.shape[0], 1))
    prediction_note = int(prediction[0][0] * 127)  # undo normalization
    prediction_output.append(prediction_note)

    # Append predicted note to pattern, remove first note
    pattern = np.append(pattern[1:], prediction_note / 127)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


In [ ]:
from music21 import stream, note

output_stream = stream.Stream()

for midi_note in prediction_output:
    n = note.Note(midi_note)
    n.quarterLength = 1
    output_stream.append(n)

output_stream.write('midi', fp='generated_song.mid')


'generated_song.mid'

In [ ]:
from google.colab import files

files.download('generated_song.mid')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>